In [1]:
################################################################################
# UNIVERSIDADE FEDERAL DE CATALÃO (UFCAT)
# WANDERLEI MALAQUIAS PEREIRA JUNIOR,                  ENG. CIVIL / PROF (UFCAT)
# MATHEUS HENRIQUE MORATO DE MORAES                    ENG. CIVIL / PROF (UFCAT)
# GUSTAVO GONÇALVES COSTA,                                    ENG. CIVIL (UFCAT)
################################################################################

################################################################################
# DESCRIÇÃO ALGORITMO:
# BIBLIOTECA DE PERDAS DE PROTENSÃO DESENVOLVIDA PELO GRUPO DE PESQUISA E ESTU-
# DOS EM ENGENHARIA (GPEE)
################################################################################


################################################################################
# BIBLIOTECAS NATIVAS PYTHON
import numpy as np

################################################################################
# BIBLIOTECAS DESENVOLVEDORES GPEE

def PERDA_DESLIZAMENTO_ANCORAGEM(P_IT0, SIGMA_PIT0, A_SCP, L_0, DELTA_ANC, E_SCP):
    """
    Esta função determina a perda de protensão por deslizamento da armadura na anco-
    ragem.
    
    Entrada:
    P_IT0       | Carga inicial de protensão                        | kN    | float
    SIGMA_PIT0  | Tensão inicial de protensão                       | kN/m² | float
    A_SCP       | Área de total de armadura protendida              | m²    | float
    L_0         | Comprimento da pista de protensão                 | m     | float
    DELTA_ANC   | Previsão do deslizamento do sistema de ancoragem  | m     | float
    E_SCP       | Módulo de Young do aço protendido                 | kN/m² | float

    Saída:
    DELTAPERC   | Perda percentual de protensão                     | %     | float
    P_IT1       | Carga final de protensão                          | kN    | float
    SIGMA_PIT1  | Tensão inicial de protensão                       | kN/m² | float
    """
    # Pré-alongamento do cabo
    DELTAL_P = L_0 * (SIGMA_PIT0 / E_SCP)
    # Redução da deformação na armadura de protensão
    DELTAEPSILON_P = DELTA_ANC / (L_0 +  DELTAL_P)
    # Perdas de protensão
    DELTASIGMA = E_SCP * DELTAEPSILON_P
    SIGMA_PIT1 = SIGMA_PIT0 - DELTASIGMA
    DELTAP = DELTASIGMA * A_SCP
    P_IT1 = P_IT0 - DELTAP
    DELTAPERC = (DELTAP / P_IT0) * 100
    return DELTAPERC, P_IT1, SIGMA_PIT1

def PERDA_DEFORMACAO_CONCRETO(E_SCP, E_CCP, P_IT0, SIGMA_PIT0, A_C, I_C, E_P, M_GPP):
    """
    Esta função determina a perda de protensão devido a deformação inicial do concreto. 
    
    Entrada:
    E_SCP       | Módulo de Young do aço protendido                 | kN/m² | float
    E_CCP       | Módulo de Young do concreto                       | kN/m² | float
    P_IT0       | Carga inicial de protensão                        | kN    | float
    SIGMA_PIT0  | Tensão inicial de protensão                       | kN/m² | float
    A_C         | Área bruta da seção                               | m²    | float 
    I_C         | Inércia da seção bruta                            | m^4   | float
    E_P         | Excentricidade de protensão                       | m     | float 
    M_GPP       | Momento fletor devido ao peso próprio             | kN.m  | float 
      
    Saída:
    DELTAPERC   | Perda percentual de protensão                     | %     | float
    P_IT1       | Carga final de protensão                          | kN    | float
    SIGMA_PIT1  | Tensão inicial de protensão                       | kN/m² | float
    """
    # Perdas de protensão
    ALPHA_P = E_SCP / E_CCP
    AUX_0 = P_IT0 / A_C
    AUX_1 = (P_IT0 * E_P ** 2) / I_C
    print(P_IT0 * E_P)
    AUX_2 = (M_GPP * E_P) / I_C
    DELTASIGMA = ALPHA_P * (AUX_0 + AUX_1 - AUX_2)
    print(DELTASIGMA)
    SIGMA_PIT1 = SIGMA_PIT0 - DELTASIGMA
    DELTAP = DELTASIGMA * A_SCP
    P_IT1 = P_IT0 - DELTAP
    DELTAPERC = (DELTAP / P_IT0) * 100
    return DELTAPERC, P_IT1, SIGMA_PIT1

def INTERPOLADOR (X_1, X_2, X_K, Y_1, Y_2):
    """
    Esta função interpola lineramente valores.

    Entrada:
    X_1   | Valor inferior X_K     |       | float
    X_2   | Valor superior X_K     |       | float
    Y_1   | Valor inferior Y_K     |       | float
    Y_2   | Valor superior Y_K     |       | float
    X_K   | Valor X de referência  |       | float

    Saída:
    Y_K   | Valor interpolado Y    |       | float
    """
    Y_K = Y_1 + (X_K - X_1) * ((Y_2 - Y_1) / (X_2 - X_1))
    return Y_K 

def TABELA_PSI1000(TIPO_FIO_CORD_BAR, TIPO_ACO, RHO_SIGMA):
    """
    Esta função encontra o fator Psi 1000 para cálculo da relaxação.

    Entrada:
    TIPO_FIO_CORD_BAR  | Tipo de armadura de protensão de acordo com a aderência escolhida                 |       | string
                       |    'FIO' - Fio                                                                    |       |
                       |    'COR' - Cordoalha                                                              |       |
                       |    'BAR' - BARRA                                                                  |       |
    TIPO_ACO           | Tipo de aço                                                                       |       | string
                       |    'RN' - Relaxação normal                                                        |       |
                       |    'RB' - Relaxação baixa                                                         |       |
    RHO_SIGMA          | Razão entre F_PK e SIGMA_PI                                                       |       | float

    Saída:
    PSI_1000           | Valor médio da relaxação, medidos após 1.000 h, à temperatura constante de 20 °C  | %     | float     
    """
    # Cordoalhas
    if TIPO_FIO_CORD_BAR == 'COR':
        if TIPO_ACO == 'RN':
            if RHO_SIGMA <= 0.5:
                PSI_1000 = 0
            elif 0.5 < RHO_SIGMA and RHO_SIGMA <= 0.6:
                Y_0 = 0.00; Y_1 = 3.50
                X_0 = 0.50; X_1 = 0.60
                X_K = RHO_SIGMA
                PSI_1000 = INTERPOLADOR(X_0, X_1, X_K, Y_0, Y_1)
            elif 0.6 < RHO_SIGMA and RHO_SIGMA <= 0.7:
                Y_0 = 3.50; Y_1 = 7.00
                X_0 = 0.60; X_1 = 0.70
                X_K = RHO_SIGMA
                PSI_1000 = INTERPOLADOR(X_0, X_1, X_K, Y_0, Y_1)
            elif 0.7 < RHO_SIGMA and RHO_SIGMA <= 0.8:
                Y_0 = 7.00; Y_1 = 12.00
                X_0 = 0.70; X_1 = 0.80
                X_K = RHO_SIGMA 
                PSI_1000 = INTERPOLADOR(X_0, X_1, X_K, Y_0, Y_1)   
        elif TIPO_ACO == 'RB':
            if RHO_SIGMA <= 0.5:
                PSI_1000 = 0
            elif 0.5 < RHO_SIGMA and RHO_SIGMA <= 0.6:
                Y_0 = 0.00; Y_1 = 1.30
                X_0 = 0.50; X_1 = 0.60
                X_K = RHO_SIGMA
                PSI_1000 = INTERPOLADOR(X_0, X_1, X_K, Y_0, Y_1)
            elif 0.6 < RHO_SIGMA and RHO_SIGMA <= 0.7:
                Y_0 = 1.30; Y_1 = 2.50
                X_0 = 0.60; X_1 = 0.70
                X_K = RHO_SIGMA
                PSI_1000 = INTERPOLADOR(X_0, X_1, X_K, Y_0, Y_1)
            elif 0.7 < RHO_SIGMA and RHO_SIGMA <= 0.8:
                Y_0 = 2.50; Y_1 = 3.50
                X_0 = 0.70; X_1 = 0.80
                X_K = RHO_SIGMA
                PSI_1000 = INTERPOLADOR(X_0, X_1, X_K, Y_0, Y_1)
    # Fio
    elif TIPO_FIO_CORD_BAR == 'FIO':
        if TIPO_ACO == 'RN':
            if RHO_SIGMA <= 0.5:
                PSI_1000 = 0
            elif 0.5 < RHO_SIGMA and RHO_SIGMA <= 0.6:
                Y_0 = 0.00; Y_1 = 2.50
                X_0 = 0.50; X_1 = 0.60
                X_K = RHO_SIGMA
                PSI_1000 = INTERPOLADOR(X_0, X_1, X_K, Y_0, Y_1)
            elif 0.6 < RHO_SIGMA and RHO_SIGMA <= 0.7:
                Y_0 = 2.50; Y_1 = 5.00
                X_0 = 0.60; X_1 = 0.70
                X_K = RHO_SIGMA
                PSI_1000 = INTERPOLADOR(X_0, X_1, X_K, Y_0, Y_1)
            elif 0.7 < RHO_SIGMA and RHO_SIGMA <= 0.8:
                Y_0 = 5.00; Y_1 = 8.50
                X_0 = 0.70; X_1 = 0.80
                X_K = RHO_SIGMA   
                PSI_1000 = INTERPOLADOR(X_0, X_1, X_K, Y_0, Y_1) 
        elif TIPO_ACO == 'RB':
            if RHO_SIGMA <= 0.5:
                PSI_1000 = 0 
            elif 0.5 < RHO_SIGMA and RHO_SIGMA <= 0.6:
                Y_0 = 0.00; Y_1 = 1.00
                X_0 = 0.50; X_1 = 0.60
                X_K = RHO_SIGMA
                PSI_1000 = INTERPOLADOR(X_0, X_1, X_K, Y_0, Y_1)
            elif 0.6 < RHO_SIGMA and RHO_SIGMA <= 0.7:
                Y_0 = 1.00; Y_1 = 2.00
                X_0 = 0.60; X_1 = 0.70
                X_K = RHO_SIGMA
                PSI_1000 = INTERPOLADOR(X_0, X_1, X_K, Y_0, Y_1)
            elif 0.7 < RHO_SIGMA and RHO_SIGMA <= 0.8:
                Y_0 = 2.00; Y_1 = 3.00
                X_0 = 0.70; X_1 = 0.80
                X_K = RHO_SIGMA  
                PSI_1000 = INTERPOLADOR(X_0, X_1, X_K, Y_0, Y_1)  
    # Barra
    elif TIPO_FIO_CORD_BAR == 'BAR':
        if RHO_SIGMA <= 0.5:
                PSI_1000 = 0 
        elif 0.5 < RHO_SIGMA and RHO_SIGMA <= 0.6:
                Y_0 = 0.00; Y_1 = 1.50
                X_0 = 0.50; X_1 = 0.60
                X_K = RHO_SIGMA
                PSI_1000 = INTERPOLADOR(X_0, X_1, X_K, Y_0, Y_1) 
        elif 0.6 < RHO_SIGMA and RHO_SIGMA <= 0.7:
                Y_0 = 1.50; Y_1 = 4.00
                X_0 = 0.60; X_1 = 0.70
                X_K = RHO_SIGMA
                PSI_1000 = INTERPOLADOR(X_0, X_1, X_K, Y_0, Y_1) 
        elif 0.7 < RHO_SIGMA and RHO_SIGMA <= 0.8:
                Y_0 = 4.00; Y_1 = 7.00
                X_0 = 0.70; X_1 = 0.80
                X_K = RHO_SIGMA 
                PSI_1000 = INTERPOLADOR(X_0, X_1, X_K, Y_0, Y_1)        
    return PSI_1000  

def PERDA_RELAXACAO_ARMADURA(P_IT0, SIGMA_PIT0, T_0, T_1, TEMP, F_PK, A_SCP, TIPO_FIO_CORD_BAR, TIPO_ACO):
    """
    Esta função determina a perda de protensão por relaxação da armadura de protensão
    em peças de concreto protendido. 
    
    Entrada:
    P_IT0              | Carga inicial de protensão                                         | kN    | float
    SIGMA_PIT0         | Tensão inicial de protensão                                        | kN/m² | float
    T_0                | Tempo inicial de análise sem correção da temperatura               | dias  | float
    T_1                | Tempo final de análise sem correção da temperatura                 | dias  | float 
    TEMP               | Temperatura de projeto                                             | °C    | float 
    F_PK               | Tensão última do aço                                               | kN/m² | float
    A_SCP              | Área de total de armadura protendida                               | m²    | float
    TIPO_FIO_CORD_BAR  | Tipo de armadura de protensão de acordo com a aderência escolhida  |       | string
                       |    'FIO' - Fio                                                     |       |
                       |    'COR' - Cordoalha                                               |       |
                       |    'BAR' - BARRA                                                   |       |
    TIPO_ACO           | Tipo de aço                                                        |       | string
                       |    'RN' - Relaxação normal                                         |       |
                       |    'RB' - Relaxação baixa                                          |       |
      
    Saída:
    DELTAPERC          | Perda percentual de protensão                                      | %     | float
    P_IT1              | Carga final de protensão                                           | kN    | float
    SIGMA_PIT1         | Tensão inicial de protensão                                        | kN/m² | float
    """
    # Determinação PSI_1000
    RHO_SIGMA = SIGMA_PIT0 / F_PK 
    if T_1 > (20 * 365):  
          PSI_1000 = 2.5
    else:
          PSI_1000 = TABELA_PSI1000(TIPO_FIO_CORD_BAR, TIPO_ACO, RHO_SIGMA)
          
    # Determinação do PSI no intervalo de tempo T_1 - T_0
    DELTAT_COR = (T_1 - T_0) * TEMP / 20
    PSI =  PSI_1000 * (DELTAT_COR / 41.67) ** 0.15
    # Perdas de protensão
    DELTASIGMA = (PSI / 100) * SIGMA_PIT0
    SIGMA_PIT1 = SIGMA_PIT0 - DELTASIGMA
    DELTAP = DELTASIGMA * A_SCP
    P_IT1 = P_IT0 - DELTAP
    DELTAPERC = (DELTAP / P_IT0) * 100
    return DELTAPERC, P_IT1, SIGMA_PIT1


In [2]:
SIGMA_PIT0 = 1453E3
A_SCP = 6.08 / 1E4
P_IT0 = SIGMA_PIT0 * A_SCP
L_0 = 150
DELTA_ANC = 6 / 1E3
E_SCP = 200E6
DELTAPERC, P_IT1, SIGMA_PIT1 = PERDA_DESLIZAMENTO_ANCORAGEM(P_IT0, SIGMA_PIT0, A_SCP, L_0, DELTA_ANC, E_SCP)
print(DELTAPERC, P_IT1, SIGMA_PIT1)

0.5466138469606744 878.5950820886262 1445057.7008036615


In [3]:
T_0 = 0; T_1 = 1; TEMP = 20; F_PK = 1900E3
TIPO_FIO_CORD_BAR = 'COR'; TIPO_ACO = 'RB'
DELTAPERC, P_IT2, SIGMA_PIT2 = PERDA_RELAXACAO_ARMADURA(P_IT1, SIGMA_PIT1, T_0, T_1, TEMP, F_PK, A_SCP, TIPO_FIO_CORD_BAR, TIPO_ACO)
print(DELTAPERC, P_IT2, SIGMA_PIT2)

1.7748733018042064 863.0011325456704 1419409.7574764316


In [4]:
E_CCP = 28000E3
A_C = 0.1245; I_C = 7.097 / 1E3; E_P = 0.30
M_GPP = 155.50
[DELTAPERC, P_IT3, SIGMA_PIT3] = PERDA_DEFORMACAO_CONCRETO(E_SCP, E_CCP, P_IT2, SIGMA_PIT2, A_C, I_C, E_P, M_GPP)
print(DELTAPERC, P_IT3, SIGMA_PIT3)

258.90033976370114
80732.93781297505
5.687782360782847 813.9155063553816 1338676.8196634564
